In [34]:
import numpy as np
import pandas as pd
import re

wordleTweets = pd.read_csv("../rawdata/tweets.csv")

wordleTweets.head(20)

wordleTweets.tail()

,wordle_id,tweet_id,tweet_date,tweet_username,tweet_text
410182,291,1511736826435878913,2022-04-06 16:05:20+00:00,SpottedLights,Wordle 291 4/6\n\n🟨⬛⬛⬛⬛\n⬛⬛🟨⬛⬛\n🟩⬛🟨🟨⬛\n🟩🟩🟩🟩🟩
410183,291,1511736827195011072,2022-04-06 16:05:20+00:00,AlisonB916,Wordle 291 4/6\n\n⬜🟩⬜⬜⬜\n⬜⬜🟨⬜⬜\n🟩🟩⬜⬜🟩\n🟩🟩🟩🟩🟩
410184,291,1511736850741878784,2022-04-06 16:05:26+00:00,Nico19596830,Wordle 291 4/6\n\n🟩⬛⬛🟨🟨\n🟩🟨🟩⬛⬛\n⬛🟩⬛⬛⬛\n🟩🟩🟩🟩🟩
410185,291,1511736856433631233,2022-04-06 16:05:27+00:00,RumzHere,Wordle 291 3/6\n\n🟩⬜🟨🟩⬜\n🟩🟨🟩⬜🟨\n🟩🟩🟩🟩🟩
410186,291,1511736863987490826,2022-04-06 16:05:29+00:00,weboggle,Wordle 291 4/6\n\n⬜⬜⬜🟨⬜\n🟩🟩⬜⬜⬜\n🟩🟩⬜⬜🟩\n🟩🟩🟩🟩🟩\n🌃


In [5]:
#sort them by wordle id to make prevent out of sequence errors
wordleTweets.sort_values(by=['wordle_id'], inplace=True)

games = []
daily = []
cur_id = 0

#process each tweet to make text machine readable
for index, tweet in wordleTweets.iterrows():
    
    #Set first index
    if index == 0:
        cur_id = tweet.wordle_id
        
    #find all guess emoju strings
    raw_guesses = re.findall("([🟩🟨⬛⬜]+)", tweet.tweet_text)
    single_game = []
    
    #replace emojs with letter
    for guess in raw_guesses:
        guess = guess.replace("🟩", 'G')
        guess = guess.replace("🟨", 'Y')
        guess = guess.replace("⬛", 'B')
        guess = guess.replace("⬜", 'B')
        single_game.append(guess)
    #print(single_game)
    
    #add to the days games
    daily.append(single_game[:])
    
    #add the day to the list and start a new daily
    if tweet.wordle_id != cur_id:
        print(len(daily))
        games.append(daily[:])
        daily.clear()
        cur_id = tweet.wordle_id

print("Complete!")

Complete!


In [ ]:
# Used for checking the data
for game in games:
    print(len(game))

In [61]:
word_freqs = pd.read_csv("../rawdata/unigram_freq.csv")

print(word_freqs.describe(),'\n')

print(word_freqs.isnull().any(), '\n')

word_freqs.dropna(inplace=True)

print(word_freqs.isnull().any(), '\n')

word_freqs.head()

              count
count  3.333330e+05
mean   1.764374e+06
std    6.629987e+07
min    1.271100e+04
25%    2.122400e+04
50%    4.151900e+04
75%    1.365760e+05
max    2.313585e+10 

word      True
count    False
dtype: bool 

word     False
count    False
dtype: bool 



,word,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698


In [72]:
answers_file = open("../rawdata/answers.txt")
raw_answers = answers_file.read()
answers = raw_answers.splitlines()
#print(answers[:10])

other_words_file = open("../rawdata/other_words.txt")
raw_other_words = other_words_file.read()
other_words = raw_other_words.splitlines()
#print(other_words[:10])

words = sorted(answers + other_words)

In [28]:
import itertools

tiles = ['G', 'Y', 'B']

raw_tile_combos = []
for raw_answer in itertools.product(tiles, repeat=5):
    raw_tile_combos.append(''.join(raw_answer))

impossibles = ['GGGGG', 'GGGGY', 'GGGYG', 'GGYGG', 'GYGGG', 'YGGGG', ]

#tile_combinations = raw_tile_combos

#for combo in impossibles:
#    tile_combinations.remove(combo)
 
tile_combinations = [BBBBB]
#print (tile_combinations)

In [94]:
#Takes in a word and returns a distribution of probably answers based on how many words correspond to a particular tile response
word_size = 5
new_line = '\n'
GREEN = 'G'
YELLOW = 'Y'
BLACK = 'B'

def get_possible_answers(word, tiles):
    re_exp = ""
    remaining_words_string = new_line.join(words)
    remaining_words_list = words
    print(len(remaining_words))
    print(tiles)
    for i in range(word_size):
        if tiles[i] == GREEN:
            for j in range(word_size):
                if i == j:
                    re_exp += word[i]
                if i != j:
                    re_exp += '.'
        if tiles[i] == YELLOW:
            #more_re = r'(?=[a-z]*[' + word[i] + '])[a-z]{5}'
            #remaining_words_list = re.findall(more_re, remaining_words_string)
            #remaining_words_string = new_line.join(remaining_words_list)
            for j in range(word_size):
                if j == 0:
                    re_exp += '^'
                if i == j:
                    re_exp += '[^' + word[i] + ']'
                if i != j:
                    re_exp += '.'
        if tiles[i] == BLACK:
            for j in range(word_size):
                if j == 0:
                    re_exp += '^'
                if i == j:
                    re_exp += '[^' + word[i] + ']'
                if i != j:
                    re_exp += '.'
        
        print(re_exp)
        remaining_words_list = re.findall(re_exp, remaining_words_string)
        remaining_words_string = new_line.join(remaining_words_list)
        print(remaining_words_list)
        re_exp = ''
    #print(remaining_words_list)
    return remaining_words_list
    
    
def get_guess_distro(word):
    answer_dict = {}
    for combo in tile_combinations:
        remaining_possibilities = get_possible_answers(word, combo)
        answer_dict[combo] = remaining_possibilities
    return answer_dict


get_guess_distro('blame')
    
    
    
    

12972
GGGGB
b....
['baaed', 'baals', 'babas', 'babel', 'babes', 'babka', 'baboo', 'babul', 'babus', 'bacca', 'bacco', 'baccy', 'bacha', 'bachs', 'backs', 'bacon', 'baddy', 'badge', 'badly', 'baels', 'baffs', 'baffy', 'bafts', 'bagel', 'baggy', 'baghs', 'bagie', 'bahts', 'bahus', 'bahut', 'bails', 'bairn', 'baisa', 'baith', 'baits', 'baiza', 'baize', 'bajan', 'bajra', 'bajri', 'bajus', 'baked', 'baken', 'baker', 'bakes', 'bakra', 'balas', 'balds', 'baldy', 'baled', 'baler', 'bales', 'balks', 'balky', 'balls', 'bally', 'balms', 'balmy', 'baloo', 'balsa', 'balti', 'balun', 'balus', 'bambi', 'banak', 'banal', 'banco', 'bancs', 'banda', 'bandh', 'bands', 'bandy', 'baned', 'banes', 'bangs', 'bania', 'banjo', 'banks', 'banns', 'bants', 'bantu', 'banty', 'banya', 'bapus', 'barbe', 'barbs', 'barby', 'barca', 'barde', 'bardo', 'bards', 'bardy', 'bared', 'barer', 'bares', 'barfi', 'barfs', 'barge', 'baric', 'barks', 'barky', 'barms', 'barmy', 'barns', 'barny', 'baron', 'barps', 'barra', 'barre', 

...m.
[]
^....[^e]
[]
12972
YGBGB
^[^b]....
['aahed']
.l...
[]
^..[^a]..
[]
...m.
[]
^....[^e]
[]
12972
YGBYG
^[^b]....
['aahed']
.l...
[]
^..[^a]..
[]
^...[^m].
[]
....e
[]
12972
YGBYY
^[^b]....
['aahed']
.l...
[]
^..[^a]..
[]
^...[^m].
[]
^....[^e]
[]
12972
YGBYB
^[^b]....
['aahed']
.l...
[]
^..[^a]..
[]
^...[^m].
[]
^....[^e]
[]
12972
YGBBG
^[^b]....
['aahed']
.l...
[]
^..[^a]..
[]
^...[^m].
[]
....e
[]
12972
YGBBY
^[^b]....
['aahed']
.l...
[]
^..[^a]..
[]
^...[^m].
[]
^....[^e]
[]
12972
YGBBB
^[^b]....
['aahed']
.l...
[]
^..[^a]..
[]
^...[^m].
[]
^....[^e]
[]
12972
YYGGG
^[^b]....
['aahed']
^.[^l]...
['aahed']
..a..
[]
...m.
[]
....e
[]
12972
YYGGY
^[^b]....
['aahed']
^.[^l]...
['aahed']
..a..
[]
...m.
[]
^....[^e]
[]
12972
YYGGB
^[^b]....
['aahed']
^.[^l]...
['aahed']
..a..
[]
...m.
[]
^....[^e]
[]
12972
YYGYG
^[^b]....
['aahed']
^.[^l]...
['aahed']
..a..
[]
^...[^m].
[]
....e
[]
12972
YYGYY
^[^b]....
['aahed']
^.[^l]...
['aahed']
..a..
[]
^...[^m].
[]
^....[^e]
[]
12972
YYGYB
^[^

['aahed']
...m.
[]
....e
[]
12972
BBYGY
^[^b]....
['aahed']
^.[^l]...
['aahed']
^..[^a]..
['aahed']
...m.
[]
^....[^e]
[]
12972
BBYGB
^[^b]....
['aahed']
^.[^l]...
['aahed']
^..[^a]..
['aahed']
...m.
[]
^....[^e]
[]
12972
BBYYG
^[^b]....
['aahed']
^.[^l]...
['aahed']
^..[^a]..
['aahed']
^...[^m].
['aahed']
....e
[]
12972
BBYYY
^[^b]....
['aahed']
^.[^l]...
['aahed']
^..[^a]..
['aahed']
^...[^m].
['aahed']
^....[^e]
['aahed']
12972
BBYYB
^[^b]....
['aahed']
^.[^l]...
['aahed']
^..[^a]..
['aahed']
^...[^m].
['aahed']
^....[^e]
['aahed']
12972
BBYBG
^[^b]....
['aahed']
^.[^l]...
['aahed']
^..[^a]..
['aahed']
^...[^m].
['aahed']
....e
[]
12972
BBYBY
^[^b]....
['aahed']
^.[^l]...
['aahed']
^..[^a]..
['aahed']
^...[^m].
['aahed']
^....[^e]
['aahed']
12972
BBYBB
^[^b]....
['aahed']
^.[^l]...
['aahed']
^..[^a]..
['aahed']
^...[^m].
['aahed']
^....[^e]
['aahed']
12972
BBBGG
^[^b]....
['aahed']
^.[^l]...
['aahed']
^..[^a]..
['aahed']
...m.
[]
....e
[]
12972
BBBGY
^[^b]....
['aahed']
^.[^l]...
['

{'GGGGB': [],
 'GGGYY': ['blabs'],
 'GGGYB': ['blabs'],
 'GGGBG': [],
 'GGGBY': ['blabs'],
 'GGGBB': ['blabs'],
 'GGYGY': [],
 'GGYGB': [],
 'GGYYG': [],
 'GGYYY': [],
 'GGYYB': [],
 'GGYBG': [],
 'GGYBY': [],
 'GGYBB': [],
 'GGBGG': [],
 'GGBGY': [],
 'GGBGB': [],
 'GGBYG': [],
 'GGBYY': [],
 'GGBYB': [],
 'GGBBG': [],
 'GGBBY': [],
 'GGBBB': [],
 'GYGGY': [],
 'GYGGB': [],
 'GYGYG': [],
 'GYGYY': ['baaed'],
 'GYGYB': ['baaed'],
 'GYGBG': [],
 'GYGBY': ['baaed'],
 'GYGBB': ['baaed'],
 'GYYGG': [],
 'GYYGY': [],
 'GYYGB': [],
 'GYYYG': [],
 'GYYYY': [],
 'GYYYB': [],
 'GYYBG': [],
 'GYYBY': [],
 'GYYBB': [],
 'GYBGG': [],
 'GYBGY': [],
 'GYBGB': [],
 'GYBYG': [],
 'GYBYY': [],
 'GYBYB': [],
 'GYBBG': [],
 'GYBBY': [],
 'GYBBB': [],
 'GBGGG': [],
 'GBGGY': [],
 'GBGGB': [],
 'GBGYG': [],
 'GBGYY': ['baaed'],
 'GBGYB': ['baaed'],
 'GBGBG': [],
 'GBGBY': ['baaed'],
 'GBGBB': ['baaed'],
 'GBYGG': [],
 'GBYGY': [],
 'GBYGB': [],
 'GBYYG': [],
 'GBYYY': [],
 'GBYYB': [],
 'GBYBG': [],
 'GBYB

tr(?=[a-z]*[al])(?![a-z]*[h])[a-z]{3}


In [74]:
#word = 'grape'
#i = 2
#more_re = '(?![a-z]*[' + word[i] + '])[a-z]{5})'
#print (more_re)
#contains = 'al'
#contains_re = '(?=[a-z]*[{0}])'.format(contains)
#disallowed = 'h'
#disallowed_re = '(?![a-z]*[{0}])'.format(disallowed)
#match = 'tr'
#word_size = 5
#match_re = '[a-z]{'+ str(word_size - len(match)) + '}'
#final_re = match + contains_re + disallowed_re + match_re
#print(final_re)

In [109]:
new_line = '\n'
remaining_words = new_line.join(words)
#print (remaining_words)
more_re = r'^[^b]....$'
results = re.findall(more_re, remaining_words)
print(len(results))
print(results)
#print(new_line.join(results))



0
[]
